In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

# 데이터 불러오기
data = pd.read_csv(r'/광진구_최종.csv', encoding='CP949')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1539 entries, 0 to 1538
Data columns (total 63 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   기준_년분기_코드        1539 non-null   int64  
 1   행정동_코드           1539 non-null   int64  
 2   행정동_코드_명         1539 non-null   object 
 3   서비스_업종_코드        1539 non-null   object 
 4   서비스_업종_코드_명      1539 non-null   object 
 5   당월_매출_금액         1539 non-null   int64  
 6   당월_매출_건수         1539 non-null   int64  
 7   주중_매출_금액         1539 non-null   int64  
 8   주말_매출_금액         1539 non-null   int64  
 9   월요일_매출_금액        1539 non-null   int64  
 10  화요일_매출_금액        1539 non-null   int64  
 11  수요일_매출_금액        1539 non-null   int64  
 12  목요일_매출_금액        1539 non-null   int64  
 13  금요일_매출_금액        1539 non-null   int64  
 14  토요일_매출_금액        1539 non-null   int64  
 15  일요일_매출_금액        1539 non-null   int64  
 16  시간대_00~06_매출_금액  1539 non-null   int64  
 17  시간대_06~11_매출_금

In [14]:
# feature, target 분리
feature = data.drop(columns=['기준_년분기_코드','행정동_코드','행정동_코드_명','서비스_업종_코드','서비스_업종_코드_명','폐업_률','1년 생존율','3년 생존율','5년 생존율','주중_매출_금액','주말_매출_금액','주중_매출_건수','주말_매출_건수'],axis=1)
target = data['폐업_률']

# train, test 분리
X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size = 0.3, random_state = 2024)

# SimpleImputer를 사용하여 결측치 대체
imputer = SimpleImputer(strategy='mean')  # 평균값으로 대체
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

In [15]:
from sklearn.model_selection import GridSearchCV

# alpha 값 후보 리스트
alphas = np.logspace(-4, 4, 10)

# GridSearchCV를 사용하여 최적의 alpha 값 찾기
lasso_cv = Lasso()
param_grid = {'alpha': alphas}
grid_search = GridSearchCV(lasso_cv, param_grid, cv=5)
grid_search.fit(X_train_imputed, Y_train)

# 최적의 alpha 값
best_alpha = grid_search.best_params_['alpha']
print("최적의 alpha:", best_alpha)

# 최적의 alpha 값으로 라쏘 모델 구축
lasso = Lasso(alpha=best_alpha, max_iter=1000).fit(X_train_imputed, Y_train)

# 회귀계수가 0이 아닌 특성의 인덱스를 찾음
non_zero_coef_indices = [i for i, coef in enumerate(lasso.coef_) if coef != 0]

# 인덱스를 사용하여 원래 feature의 이름을 가져옴
feature_list = feature.columns[non_zero_coef_indices]

print("선택된 특성:", feature_list)

print(lasso.coef_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.710e+03, tolerance: 2.455e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.702e+03, tolerance: 2.451e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.124e+03, tolerance: 2.676e

최적의 alpha: 0.3593813663804626
선택된 특성: Index(['당월_매출_금액', '당월_매출_건수', '월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액',
       '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액',
       '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액',
       '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액', '여성_매출_금액',
       '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액',
       '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '월요일_매출_건수', '화요일_매출_건수',
       '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수',
       '시간대_건수~06_매출_건수', '시간대_건수~11_매출_건수', '시간대_건수~14_매출_건수',
       '시간대_건수~17_매출_건수', '시간대_건수~21_매출_건수', '시간대_건수~24_매출_건수', '남성_매출_건수',
       '여성_매출_건수', '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수',
       '연령대_40_매출_건수', '연령대_50_매출_건수', '연령대_60_이상_매출_건수', '점포_수', '유사_업종_점포_수',
       '개업_율', '개업_점포_수', '폐업_점포_수'],
      dtype='object')
[-9.29332535e-10  8.30477353e-06 -6.03786402e-10 -1.92898318e-09
  2.70458641e-09 -2.34485428e-09  4.03518001e-10 -1.104353

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.002e+04, tolerance: 3.127e+00
  model = cd_fast.enet_coordinate_descent(
